# Catalog Listener

The `CatalogListener` class provides methods for listening to an FDSN (or similar) client for new events. These events can be used to generate
template which can be stored in the provided `TemplateBank`. The `CatalogListener` is a subclass of the `_Listener` abstract base class. Some
event services provide a "push" service for event updates, and you could write your own listener (that subclasses the `_Listener` ABC) that 
reacts to push events.  The `CatalogListener` operates on a "pull" basis: it regularly queries an event provider for updates.

For example, we can set-up a `CatalogListener` to update our `TemplateBank` based on the New Zealand GeoNet catalog. Note that we provide a seperate
`waveform_client` because GeoNet provides [recent data through a different](https://www.geonet.org.nz/data/tools/FDSN), but incomplete, service - 
if your client has near-real time data available via their main FDSN routing you do not need to provide a seperate `waveform_client`.

In [1]:
from obspy.clients.fdsn import Client
from obspy import Catalog
from rt_eqcorrscan.event_trigger import CatalogListener
from rt_eqcorrscan.database import TemplateBank

template_bank = TemplateBank(base_path="./listening_db")
listener = CatalogListener(
    client=Client("GEONET"), waveform_client=Client("http://service-nrt.geonet.org.nz"),
    template_bank=template_bank, interval=5)
print(listener)

CatalogListener(client=Client(http://service.geonet.org.nz), catalog=Catalog(0 events), interval=5, **kwargs)


When we instantiate the listener it doesn't automatically run.  We have the option to run the listener either in the foreground, or in a background
(daemon) thread. Running it in the background allows us to do other things while the listener runs. We pass `template_kwargs` as a dictionary of
arguments supported by EQcorrscan's `template_gen` [function](https://eqcorrscan.readthedocs.io/en/latest/submodules/autogen/eqcorrscan.core.template_gen.template_gen.html#eqcorrscan.core.template_gen.template_gen).

For this example we will set the logging output level to debug to get some output while running - in higher levels this process should be nearly silent.

In [2]:
import logging

logging.basicConfig(
    level="DEBUG", format="%(asctime)s\t%(name)s\t%(levelname)s\t%(message)s")

listener.background_run(
    template_kwargs={
        "lowcut": 2.0, "highcut": 15.0, "filt_order": 4, "samp_rate": 50., "length": 4.,
        "prepick": .2, "swin": "all", "process_len": 300, "min_snr": 0,})
print(listener)

2019-07-18 13:23:38,438	rt_eqcorrscan.event_trigger.catalog_listener	DEBUG	Checking for new events between 2019-07-18T01:23:38.425452Z and 2019-07-18T01:23:38.438295Z
2019-07-18 13:23:38,438	rt_eqcorrscan.event_trigger.listener	INFO	Started listening to http://service.geonet.org.nz


CatalogListener(client=Client(http://service.geonet.org.nz), catalog=Catalog(0 events), interval=5, **kwargs)


2019-07-18 13:23:38,549	rt_eqcorrscan.event_trigger.catalog_listener	DEBUG	No new data
2019-07-18 13:23:43,555	rt_eqcorrscan.event_trigger.catalog_listener	DEBUG	Checking for new events between 2019-07-18T01:23:38.425452Z and 2019-07-18T01:23:43.555627Z
2019-07-18 13:23:43,664	rt_eqcorrscan.event_trigger.catalog_listener	DEBUG	No new data


The listener has a cache of events with it, the length of this cache is controlled by the `keep` keyword argument on instantiation.  While the
listener is running in the background we can query the cache to see what has changed.

In [3]:
print(listener.old_events)

[]


Depending on what was happening when this notebook ran, the above cell might output a list with event ids and times, or it might output an empty list.

We can stop the listener when we are done with it.

In [4]:
listener.background_stop()

This can take a while: it will wait until the next expected update before stopping.

The listener above will automatically update the `template_bank` when events are registered. You can turn this off using the `make_templates`
keyword argument on `run` and `background_run`

In [5]:
listener.background_run(make_templates=False)
print(listener)
listener.background_stop()

2019-07-18 13:23:59,710	rt_eqcorrscan.event_trigger.catalog_listener	DEBUG	Checking for new events between 2019-07-18T01:23:38.425452Z and 2019-07-18T01:23:59.710583Z
2019-07-18 13:23:59,710	rt_eqcorrscan.event_trigger.listener	INFO	Started listening to http://service.geonet.org.nz
2019-07-18 13:23:59,823	rt_eqcorrscan.event_trigger.catalog_listener	DEBUG	No new data


CatalogListener(client=Client(http://service.geonet.org.nz), catalog=Catalog(0 events), interval=5, **kwargs)
